In [ ]:
import pandas as pd

# ====== CSVのパス ======
CAT_CSV   = "csv_exports/category.csv"
FC_CSV    = "csv_exports/film_category.csv"
FILM_CSV  = "csv_exports/film.csv"
INV_CSV   = "csv_exports/inventory.csv"
RENT_CSV  = "csv_exports/rental.csv"
CUS_CSV   = "csv_exports/customer.csv"
ADDR_CSV  = "csv_exports/address.csv"
CITY_CSV  = "csv_exports/city.csv"
COUNTRY_CSV = "csv_exports/country.csv"

# ====== 国×カテゴリの件数集計 ======
def make_country_category_counts():
    cat   = pd.read_csv(CAT_CSV)
    fc    = pd.read_csv(FC_CSV)
    film  = pd.read_csv(FILM_CSV)
    inv   = pd.read_csv(INV_CSV)
    rent  = pd.read_csv(RENT_CSV, parse_dates=["rental_date"])
    cus   = pd.read_csv(CUS_CSV)
    addr  = pd.read_csv(ADDR_CSV)
    city  = pd.read_csv(CITY_CSV)
    country = pd.read_csv(COUNTRY_CSV)

    # 顧客→住所→都市→国
    cus_country = (cus.merge(addr, on="address_id", how="left")
                       .merge(city, on="city_id", how="left")
                       .merge(country, on="country_id", how="left"))

    # rental -> customer -> country
    r_cus = rent.merge(cus_country[["customer_id","country"]], on="customer_id", how="left")

    # rental -> inventory(→film) -> film_category -> category
    df = (r_cus.merge(inv[["inventory_id","film_id"]], on="inventory_id", how="left")
                 .merge(fc, on="film_id", how="left")
                 .merge(cat, on="category_id", how="left"))

    # ピボット（件数）
    pt = pd.pivot_table(df, index="country", columns="name",
                        values="rental_id", aggfunc="count", fill_value=0)

    return pt

if __name__ == "__main__":
    pt_country = make_country_category_counts()
    print("国×カテゴリの件数ピボット（先頭5行）:")
    print(pt_country.head(), "\n")

    # CSVに保存
    out_csv = "country_category_counts.csv"
    pt_country.to_csv(out_csv, encoding="utf-8-sig")
    print(f"CSVファイルに出力しました: {out_csv}")
